In [ ]:
import tifffile
from tqdm import tqdm
from pathlib import Path
from cellpose import models
import numpy as np
from skimage import measure, exposure, filters
import pandas as pd
import napari
import pyclesperanto_prototype as cle
import matplotlib.pyplot as plt

In [ ]:
# Scan for all images present in the data folder and add their Paths to the images list
data_path = Path("./data")
images = []

for file_path in data_path.glob("**/**/**/*.lsm"):
    images.append(file_path)

In [ ]:
model = models.Cellpose(gpu=True, model_type="nuclei")
cellpose_nuclei_diameter = 40
gaussian_sigma = 5
h2a_threshold = 40 # Mean intensity within the nucleus mask (h2a signal)
cfos_threshold = 20 # Mean intensity within the nucleus mask (cfos signal)

for image in images:

    # Extract filename, region, mouse and IHC round
    file_path = Path(image)
    file_path_parts = file_path.parts
    filename = file_path.stem
    region = file_path_parts[-2]
    mouse_id = file_path_parts[-3]
    ihc_round = file_path_parts[-4]

    # Read the image file
    img = tifffile.imread(image)

    # Extract per channel info
    nuclei_img = img[0,:,:]
    h2a_img = img[1,:,:]
    cfos_img = img[2,:,:]

    # Create a copy of nuclei_mip
    input_img = nuclei_img.copy()

    # Might need to perform a Gaussian-blur before
    post_gaussian_img = filters.gaussian(
        input_img, sigma=gaussian_sigma
    )

    # Apply Contrast Stretching to improve Cellpose detection of overly bright nuclei
    p2, p98 = np.percentile(post_gaussian_img, (2, 98))
    img_rescale = exposure.rescale_intensity(
        post_gaussian_img, in_range=(p2, p98)
    )

    # Predict nuclei nuclei_masks using cellpose
    nuclei_masks, flows, styles, diams = model.eval(
        img_rescale,
        diameter=cellpose_nuclei_diameter,
        channels=[0, 0],
        net_avg=False,
                )
    
    # Morphological and intensity measurements
    
    # Extract regionprops from nuclei labels and H2A channel
    h2a_props = measure.regionprops_table(
        label_image=nuclei_masks,
        intensity_image=h2a_img,
        properties=[
            "label",
            "intensity_mean",
            "intensity_max",
            "area_filled",
            "perimeter",
            "equivalent_diameter"
        ],
    )

    # Construct a dataframe
    h2a_df = pd.DataFrame(h2a_props)

    # Rename the intensity columns for further merging with cfos_df
    h2a_df.rename(columns={'intensity_mean': 'h2a_intensity_mean'}, inplace=True)
    h2a_df.rename(columns={'intensity_max': 'h2a_intensity_max'}, inplace=True)
    
    # Extract regionprops from nuclei labels and CFOS channel
    cfos_props = measure.regionprops_table(
        label_image=nuclei_masks,
        intensity_image=cfos_img,
        properties=[
            "label",
            "intensity_mean",
            "intensity_max"
        ],
    )

    # Construct a dataframe
    cfos_df = pd.DataFrame(cfos_props)

    # Rename the intensity columns for further merging with cfos_df
    cfos_df.rename(columns={'intensity_mean': 'cfos_intensity_mean'}, inplace=True)
    cfos_df.rename(columns={'intensity_max': 'cfos_intensity_max'}, inplace=True)
    
    merged_df = pd.merge(cfos_df, h2a_df, on='label', how='inner')
    
    # Create a new DataFrame with the same index as merged_df and the new columns
    new_columns_df = pd.DataFrame({
        'filename': [filename] * len(merged_df),
        'region': [region] * len(merged_df),
        'mouse_id': [mouse_id] * len(merged_df),
        'ihc_round': [ihc_round] * len(merged_df)
    }, index=merged_df.index)

    # Concatenate the new columns DataFrame with the original merged_df
    # Using pd.concat and specifying axis=1 for columns, and placing the new DataFrame first
    merged_df = pd.concat([new_columns_df, merged_df], axis=1)
    
    # Select H2A and CFOS positive cells based on mean_intensity thresholds, return a mask of + cells
    
    # Filtering the DataFrames for values above the set threshold
    h2a_filtered_df = merged_df[merged_df['h2a_intensity_mean'] > h2a_threshold]
    cfos_filtered_df = merged_df[merged_df['cfos_intensity_mean'] > cfos_threshold]

    # Extracting the h2a and cfos label values as a list
    h2a_pos_labels = h2a_filtered_df['label'].tolist()
    cfos_pos_labels = cfos_filtered_df['label'].tolist()

    # Convert lists to NumPy arrays to leverage vectorized comparisons
    h2a_pos_labels_array = np.array(h2a_pos_labels)
    cfos_pos_labels_array = np.array(cfos_pos_labels)

    # Find common labels  
    double_pos_labels = np.intersect1d(h2a_pos_labels_array, cfos_pos_labels_array)
    
    # Create copies of the original nuclei_masks array
    h2a_nuclei_labels = nuclei_masks.copy()
    cfos_nuclei_labels = nuclei_masks.copy()
    double_pos_nuclei_labels = nuclei_masks.copy()

    nuclei_labels_arrays = [h2a_nuclei_labels, cfos_nuclei_labels, double_pos_nuclei_labels]
    labels_to_keep = [h2a_pos_labels, cfos_pos_labels, double_pos_labels]

    for array, labels in zip(nuclei_labels_arrays, labels_to_keep):

        # Check if elements are in the 'values_to_keep' array
        in_values = np.isin(array, labels)

        # Set elements not in 'values_to_keep' to zero
        array[~in_values] = 0
        
    # Push the image to GPU memory
    label_image_gpu = cle.push(nuclei_masks)

    # Detect edges
    edges = cle.detect_label_edges(label_image_gpu)

    # Mask original labels with edges
    labeled_edges = label_image_gpu * edges

    # Dilate labels to make visualization easier
    dilated_labeled_edges = cle.dilate_labels(labeled_edges, radius=1)
    
    print(f"Filepath: {file_path}")
    print(f"H2A_+ labels: {h2a_pos_labels}")
    print(f"CFOS_+ labels: {cfos_pos_labels}")
    print(f"H2A & CFOS_+ labels: {double_pos_labels}")
    
    # Visualize the segmentation results on a per image basis
    plt.figure(figsize=(40, 40))
    plt.subplot(1, 7, 1)
    plt.imshow(nuclei_img, cmap='gray')
    plt.title('Input Nuclei Image')
    plt.axis("off")

    plt.subplot(1, 7, 2)
    plt.imshow(dilated_labeled_edges, cmap='viridis')
    plt.title('Segmentation nuclei')
    plt.axis("off")

    plt.subplot(1, 7, 3)
    plt.imshow(h2a_img, cmap='gray')
    plt.title('Input H2A Image')
    plt.axis("off")

    plt.subplot(1, 7, 4)
    plt.imshow(h2a_nuclei_labels, cmap='viridis')
    plt.title('H2A + cells')
    plt.axis("off")

    plt.subplot(1, 7, 5)
    plt.imshow(cfos_img, cmap='gray')
    plt.title('Input CFOS Image')
    plt.axis("off")

    plt.subplot(1, 7, 6)
    plt.imshow(cfos_nuclei_labels, cmap='viridis')
    plt.title('CFOS + cells')
    plt.axis("off")

    plt.subplot(1, 7, 7)
    plt.imshow(double_pos_nuclei_labels, cmap='viridis')
    plt.title('Double H2A/CFOS + cells')
    plt.axis("off")

    plt.show()


In [ ]:
# Initialize napari.Viewer and display input stacks and label processing steps
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(img)
viewer.add_image(nuclei_img)
viewer.add_image(post_gaussian_img)
viewer.add_image(img_rescale)
viewer.add_labels(nuclei_masks)
viewer.add_labels(h2a_nuclei_labels)
viewer.add_labels(cfos_nuclei_labels)
viewer.add_labels(double_pos_nuclei_labels)
viewer.add_labels(edges_image)